Math shared by all implementations

All implementations have the fundamental bits:

input matrix X of size [num_samples, num_features]

output matrix y of size [num_samples, num_outputs]

projection matrix W of size [num_features, num_neurons]

non-linear tranformation function ufunc like hyperbolic tangent (np.tanh), sigmoid, or any other

output weights B of size [num_neurons, num_outputs]

optional output bias/intercept of size [num_outputs, ]


In [513]:
import numpy as np
import pandas  as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from scipy import linalg
from math import sqrt

In [514]:
class clientClass:
   
    def __init__(self,client_id,batch_size,h_neurons=20, alpha=0.001):
        
        self.i=client_id
        cl= np.load(f"client{self.i}.npz")

        if client_id == 1:
            n = 100
        elif client_id == 2:
            n = 1000
        else:
            n = 10000
            
        
        X_t = cl[f"X_a{self.i}_sc"][:n]
        X_s = cl[f"X_a{self.i}_sc"][n:]
        y_t = cl[f"y_a{self.i}"][:n]
        y_s = cl[f"y_a{self.i}"][n:]

           
        self.input_size = len(X_t)
        self.n_features = len(X_t[1])
        self.h_neurons=h_neurons
        self.w = None
        self.b = None
        self.batch_size=batch_size
        self.reg_term = np.eye(self.h_neurons) * alpha
        self.X_t = X_t
        self.X_s = X_s
        self.y_t = y_t
        self.y_s = y_s
          
            
    def get_Nfs(self):
        return self.input_size, self.n_features
    
    def set_W_b(self,w,b):
        self.w=w
        self.b=b  
    
    def batches(self):
        
        batch_X=[]
        batch_y=[]
        
        if self.batch_size >= self.input_size:
            batch_X.append(self.X_t)
            batch_y.append(self.y_t)
             
        else:
            for i in range (0, self.input_size , self.batch_size):
                batch_X.append(self.X_t[i:i + self.batch_size-1])
                batch_y.append(self.y_t[i:i + self.batch_size-1])
            
        for X,y in zip(batch_X,batch_y): 
            
             # Compute the hidden layer output 
            H = np.tanh(np.dot(X, self.w) + self.b)  # Apply activation function (tanh)

            HTH = H.T @ H + self.reg_term
            HTy = H.T @ y
              
            yield HTH, HTy               
   
    def predict_test(self, B):
                                    
        # Calculate the predicted output on the testing set
        H_test = np.tanh(np.dot(self.X_s, self.w) + self.b)
        pred_test = np.dot(H_test, B)
        r2 = r2_score(self.y_s, pred_test)
        return r2


In [515]:


def par_B(cl1,cl2,cl3):
    # Perform some operation using a for loop
    # combine some client data
    clients = [cl1, cl2, cl3]
    HtH1 = None
    HtY1 = None
    Bd1={}
    B1=[]
    Bd=[]

    for client in clients:
        sample, c_nf= client.get_Nfs()
        w_inital = np.random.rand(c_nf, hidden_size)
        bias_inital = np.random.rand(hidden_size)
        #print(w_inital)

        client.set_W_b(w_inital, bias_inital)
        data = client.batches()

        for hth, hty in data:

            HtH1 = hth
            HtY1 = hty

            #print(f"Client   : {client.get_Nfs()}")
            Bd.append(np.linalg.solve(HtH1, HtY1))
            #print(f"Bd: {Bd}")
        Bd1[client.i] = Bd 
        Bd=[]

    return Bd1

In [516]:
# I am a server now
# initialize ELM parameters shared among clients

hidden_size=20

# initialize clients, server sends ELM parameters to clients
cl1 = clientClass(1, batch_size=100, h_neurons=hidden_size, alpha=0.001)
cl2 = clientClass(2, batch_size=300, h_neurons=hidden_size, alpha=0.001)
cl3 = clientClass(3, batch_size=1000, h_neurons=hidden_size, alpha=0.001)

#cl1_input_size,cl1_n_features =cl1.get_Nfs()
#cl2_input_size,cl2_n_features =cl2.get_Nfs()
#cl3_input_size,cl3_n_features =cl3.get_Nfs()

input_size=8
w_inital = np.random.rand(input_size, hidden_size)
bias_inital = np.random.rand(hidden_size)

clients = [cl1, cl2, cl3]
cl1.set_W_b(w_inital, bias_inital)
cl2.set_W_b(w_inital, bias_inital)
cl3.set_W_b(w_inital, bias_inital)


In [517]:
def average_matrices_Sl(matrix1):
    # Check if the matrices have the same shape
    if len(matrix1) == 0:
        return None  # Handle the case of an empty list
    average_matrix = sum(matrix1 ) / len(matrix1)
    return average_matrix
    

In [595]:
def global_B(B):
    ave_client123=[]
    for i in range(1,len(B)+1):
        print(f"Client_{i}")
        ave_client123.append(average_matrices_Sl(B[i]))
        print(f"{average_matrices_Sl(B_1i[i])}")
        print()
    print(f"global_B: {average_matrices_Sl(ave_client123)}")
    return average_matrices_Sl(ave_client123)



In [596]:
B_1i=par_B(cl1,cl2,cl3)

In [597]:
B_1i

{1: [array([-1.03621715,  3.35771644,  0.90983135,  0.08746734,  0.55120982,
          0.70208885,  0.28002474, -0.98793492, -1.18641596, -0.60534182,
         -0.37784304, -0.94961194, -0.43184936, -0.60853668,  1.29557916,
          0.64490891, -1.34905212, -0.00747328, -0.17215557,  0.317743  ])],
 2: [array([ 0.47924513, -2.85576855,  0.37706179,  0.84327277,  1.45157396,
          3.01046342, -2.3477088 , -3.62104037, -0.410038  ,  0.34219976,
          0.15277389,  2.58398422,  0.19015596, -1.29229265, -1.15419647,
          2.54481009,  0.04520338,  0.16167391, -0.26929891, -1.61719079]),
  array([ 0.50740823,  1.3051313 , -0.29491174, -0.70669717,  0.92440962,
         -1.33006819, -0.58108266, -0.28488152,  0.46787198,  0.58413759,
         -0.17352582,  0.00361403,  1.10680242, -0.78059342, -1.45255798,
          0.25564221,  0.10632179, -0.80470159,  0.03856042, -0.30359351]),
  array([ 0.55336842,  0.66822395,  0.96933643,  1.19457895,  0.26843029,
          0.26265511,  2.

In [598]:
g_B_it1=global_B(B_1i)

Client_1
[-1.03621715  3.35771644  0.90983135  0.08746734  0.55120982  0.70208885
  0.28002474 -0.98793492 -1.18641596 -0.60534182 -0.37784304 -0.94961194
 -0.43184936 -0.60853668  1.29557916  0.64490891 -1.34905212 -0.00747328
 -0.17215557  0.317743  ]

Client_2
[ 0.4655549   0.52550201  0.15785194  0.08040586  0.89195021  0.32741848
 -0.43500066 -0.99245356  0.13803874  2.28674895 -0.08947329  0.54622544
  0.68390393 -1.12223043 -1.31388344 -0.07109745 -1.36325258 -0.39743928
 -0.2470925  -2.12000381]

Client_3
[-0.70202661  0.18101333 -0.23195878 -0.46635196  0.31457925 -0.4307489
 -0.40509862 -0.05395757 -0.26089757  0.77029415 -1.52706743 -0.11786234
 -0.13682668 -0.06113777 -0.67724098  0.3262019   0.74306722  0.58446432
  0.58737323  0.18976582]

global_B: [-0.42422962  1.35474393  0.27857484 -0.09949292  0.5859131   0.19958615
 -0.18669151 -0.67811535 -0.43642493  0.81723376 -0.66479458 -0.17374961
  0.0384093  -0.59730163 -0.23184842  0.30000445 -0.6564125   0.05985059
  0.056

In [599]:
g_B_it1

array([-0.42422962,  1.35474393,  0.27857484, -0.09949292,  0.5859131 ,
        0.19958615, -0.18669151, -0.67811535, -0.43642493,  0.81723376,
       -0.66479458, -0.17374961,  0.0384093 , -0.59730163, -0.23184842,
        0.30000445, -0.6564125 ,  0.05985059,  0.05604172, -0.53749833])

In [583]:
r2_it1=cl1.predict_test(g_B_it1)
r2_it1

-52.453824598204484

In [584]:
r2_it2=cl2.predict_test(g_B_it1)
r2_it2

-8.64636573331588

In [575]:
r2_it3=cl3.predict_test(g_B_it1)
r2_it3

-0.46851009721826054